<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [96]:
import numpy as np
from sklearn.utils import shuffle

#import train_test_split to create training and testing sets
from sklearn.model_selection import train_test_split
def load_quickdraw10(path):
    """ Function returns train and testing matrices w/ corresponding target vectors  for given data
    returns X_train,X_test, y_train,  y_test"""
    path = np.load('quickdraw10.npz')
    data = path
    X = data['arr_0']
    y = data['arr_1']
    X, y = shuffle(X, y)
    
    return train_test_split(X, y, test_size=0.20, random_state=42)

In [97]:
X_train,X_test, y_train, y_test = load_quickdraw10('module3-Tune/quickdraw10.npz')

In [98]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((80000, 784), (80000,), (20000, 784), (20000,))

In [57]:
# import numpy and pandas libararies incase we need to modify data in pandas data frame
import numpy
import pandas as pd
#Help with hyperparameter tunning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#Import model required to build model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [58]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [59]:
#Load models required to build tensor board
#Tensorboard will help us organize our experiments ran on our model
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import os
import datetime

In [83]:
# Number of units represented number of neurons
# hp.Discrete( will pick values discretely that is either 10,15, or 30 per run 
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([10,15,30]))
# Represents how much to change the model depending on the error - value too smale may result 
# in a long training process whereas a value to large may result in learning a sub-optimal set of weights too fast
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,.01))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd','adamax']))
HP_ACTIVATION = hp.HParam('activation',hp.Discrete(['relu','softmax']))
HP_WEIGHTS = hp.HParam('kernel_initializer', hp.Discrete(['random_uniform','random_normal'])) #weights
HP_BIAS = hp.HParam('bias_initializer', hp.Discrete(['zeros','ones'])) # bias                       
#How were going to test how good our model performed
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER,HP_ACTIVATION,HP_WEIGHTS,HP_BIAS],
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )

In [84]:
def train_test_model(hparams):
    
 # construct model 
  model = tf.keras.Sequential(
      [tf.keras.layers.Dense(hparams[HP_NUM_UNITS],hparams[HP_ACTIVATION],hparams[HP_WEIGHTS],hparams[HP_BIAS]),
       tf.keras.layers.Dense(10, hparams[HP_ACTIVATION])      
  ])

  opt_name = hparams[HP_OPTIMIZER]
  lr = hparams[HP_LEARNING_RATE]

  if opt_name == 'adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
  elif opt_name == 'sgd':
    opt = tf.keras.optimizers.SGD(learning_rate=lr)
  elif opt_name == 'adamax':
    opt = tf.keras.optimizers.Adamax(learning_rate=lr)  
  else:
    raise ValueError(f'Unexpected optimizer: {opt_name}')

  model.compile(
      optimizer=opt,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )
  #train model and test it  
  model.fit(X_train, y_train, epochs=5)
  _, accuracy = model.evaluate(X_test, y_test)



  return accuracy

In [85]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [89]:
session_num = 0

# Basically a grid search
for num_units in HP_NUM_UNITS.domain.values:
  for learning_rate in (HP_LEARNING_RATE.domain.min_value,
                        HP_LEARNING_RATE.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
        for activation in HP_ACTIVATION.domain.values:
            for kernel_initializer in HP_WEIGHTS.domain.values:
                for bias_initializer in HP_BIAS.domain.values:
                  hparams = {
                  HP_NUM_UNITS: num_units,
                  HP_LEARNING_RATE: learning_rate,
                  HP_OPTIMIZER: optimizer,
                  HP_ACTIVATION: activation,
                  HP_WEIGHTS:kernel_initializer,
                  HP_BIAS:bias_initializer,
                  
                  
                       }

                  run_name = f'run-{session_num}'
                  print(f'--- Starting trial: {run_name}')
                  print({param.name: hparams[param] for param in hparams})
                  run('logs/hparams_tuning/' + run_name, hparams)
                  session_num += 1

--- Starting trial: run-0
{'num_units': 10, 'learning_rate': 0.001, 'optimizer': 'adam', 'activation': 'relu', 'kernel_initializer': 'random_normal', 'bias_initializer': 'ones'}
Epoch 1/5
2094/2094 [==============================] - 2s 786us/step - loss: 11.0615 - accuracy: 0.0986
Epoch 2/5
2094/2094 [==============================] - 2s 728us/step - loss: 11.0615 - accuracy: 0.0986
Epoch 3/5
2094/2094 [==============================] - 2s 754us/step - loss: 11.0615 - accuracy: 0.0986
Epoch 4/5
2094/2094 [==============================] - 2s 764us/step - loss: 11.0615 - accuracy: 0.0986 1s - loss: 11.0
Epoch 5/5
1032/1032 [==============================] - 0s 469us/step - loss: 11.0484 - accuracy: 0.1029
--- Starting trial: run-1
{'num_units': 10, 'learning_rate': 0.001, 'optimizer': 'adam', 'activation': 'relu', 'kernel_initializer': 'random_normal', 'bias_initializer': 'zeros'}
Epoch 1/5
2094/2094 [==============================] - 2s 756us/step - loss: 2.3026 - accuracy: 0.1008
Epoc

In [93]:
!ls

9_9_2020_LS_DS17_423_Tune_Lecture.ipynb previous_lectures
LS_DS_433_Tune_Assignment.ipynb         quickdraw10.npz
logs


In [94]:
!ls logs

hparam_tuning  hparams_tuning


In [95]:
#Visualize Trials
%tensorboard --logdir logs/hparams_tuning

Reusing TensorBoard on port 6006 (pid 7815), started 8:09:47 ago. (Use '!kill 7815' to kill it.)

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 